First the imports

In [2]:
import os 
import shutil # to copy files across directories

import pandas as pd
import numpy as np
import csv
import calendar

# Exploration & Cleaning:

If there is no CleanedData directory, then create one

In [2]:
# creates a directory CleanedData if it does not exist 
def create_cleaning_directory(clean_dir):

    if not os.path.exists(clean_dir):
        os.makedirs(clean_dir)
        print(f"Directory '{clean_dir}' created successfully.")
    else:
        print(f"Directory '{clean_dir}' already exists.")

In [ ]:
cleaning_dir = "../CleanedData"
create_cleaning_directory(cleaning_dir)

Next, we want to have the data in an accessible form. This is done by reading the data into a dictionary of panada dataframes. Each element in the dictionary has a key (which is the name of the month the data was collected in) and a value (the csv entries in a panda dataframe) 

In [3]:
# returns a dictionary containing the data from the .csv files
def read_data(dir):
    dfs = {}
    files = os.listdir(dir)

    # filter files by .csv bec some are google sheets
    # extract the number of the month, then sort by month so that they are then stored in order of month
    csv_files = sorted([int(file[4:6]) for file in files if file.endswith('.csv')])
    
    for csv_file in csv_files:
        # reconstruct the name of file bec we extracted the month number to sort them by month
        reconstructed_name = '2023'+str(csv_file)+'-divvy-tripdata.csv' if csv_file >= 10 else '20230'+str(csv_file)+'-divvy-tripdata.csv'
        f = os.path.join(dir, reconstructed_name) 
        
        # get month name
        month = calendar.month_name[csv_file]
        
        # read csv file into dataframe
        dfs[month] = pd.read_csv(f)

    return dfs

In [4]:
original_dir = "../../BikeShareData/OriginalData"
data = read_data(original_dir)

Some dataset exploration

In [50]:
no_of_files = len(data)
print(no_of_files)

11


In [51]:
month_names = data.keys()
print(month_names)

dict_keys(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November'])


In [52]:
# the first and last 5 entries
#data["January"]
#data["February"]
#data["March"]
#data["April"]
data["May"]
#data["June"]
#data["July"]
#data["August"]
#data["September"]
#data["October"]
#data["November"]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0D9FA920C3062031,electric_bike,2023-05-07 19:53:48,2023-05-07 19:58:32,Southport Ave & Belmont Ave,13229,NaN,NaN,41.939408,-87.663831,41.930000,-87.650000,member
1,92485E5FB5888ACD,electric_bike,2023-05-06 18:54:08,2023-05-06 19:03:35,Southport Ave & Belmont Ave,13229,NaN,NaN,41.939482,-87.663848,41.940000,-87.690000,member
2,FB144B3FC8300187,electric_bike,2023-05-21 00:40:21,2023-05-21 00:44:36,Halsted St & 21st St,13162,NaN,NaN,41.853793,-87.646719,41.860000,-87.650000,member
3,DDEB93BC2CE9AA77,classic_bike,2023-05-10 16:47:01,2023-05-10 16:59:52,Carpenter St & Huron St,13196,Damen Ave & Cortland St,13133,41.894556,-87.653449,41.915983,-87.677335,member
4,C07B70172FC92F59,classic_bike,2023-05-09 18:30:34,2023-05-09 18:39:28,Southport Ave & Clark St,TA1308000047,Southport Ave & Belmont Ave,13229,41.957081,-87.664199,41.939478,-87.663748,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
604822,48BDA26F34445546,electric_bike,2023-05-18 10:26:43,2023-05-18 10:48:00,Clark St & Elmdale Ave,KA1504000148,NaN,NaN,41.990876,-87.669721,42.000000,-87.660000,member
604823,573025E5EDE10DE1,electric_bike,2023-05-17 14:32:48,2023-05-17 14:45:37,State St & 33rd St,13216,NaN,NaN,41.834734,-87.625798,41.830000,-87.620000,member
604824,D88D48898C6FB63E,electric_bike,2023-05-17 07:59:29,2023-05-17 08:04:54,Columbus Dr & Randolph St,13263,NaN,NaN,41.884422,-87.619393,41.880000,-87.630000,member
604825,4692DCD2F87497F5,electric_bike,2023-05-18 08:34:48,2023-05-18 08:38:40,Public Rack - Karlov Ave & Lawrence Ave,1127.0,NaN,NaN,41.970000,-87.730000,41.970000,-87.740000,member


In [57]:
data["May"].dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [54]:
# the number of unique values in May
# unique values for specific columns
print("Column Name:          NUnique")
print(data['May'].nunique())
print(data['May'].rideable_type.unique())
print(data['May'].member_casual.unique())

Column Name:          NUnique
ride_id               604827
rideable_type              3
started_at            503683
ended_at              505259
start_station_name      1287
start_station_id        1250
end_station_name        1254
end_station_id          1210
start_lat             188591
start_lng             185410
end_lat                 4759
end_lng                 4762
member_casual              2
dtype: int64
['electric_bike' 'classic_bike' 'docked_bike']
['member' 'casual']


The count_entries method will do the following:
 * check whether or not to remove duplicates
 * calculate number of entires in each file
 * calculate number of columns in each file
 * calculate the average and total number of entries across all files
 * write these outputs to a file

In [76]:
def count_entries(dataset, file_name, flag):
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Month","No Of Entries","No Of Cols"])

        total = 0

        for month, df in dataset.items():
            if flag:
                df = df.drop_duplicates()
                
            entries = len(df)
            writer.writerow([month, entries,len(df.columns)])
            total += len(df)
            print(f"For {month}:{df.shape}")

        average = int(total/11)
        writer.writerow(["Total:", total])
        writer.writerow(["Average:", average])
        print(f"Average number of entries per file: {average}")
        print(f"Total number of entries across all files: {total}")


In [47]:
count_entries(data, "Original_BikeRides.csv", False)

For January:(190301, 13)
For February:(190445, 13)
For March:(258678, 13)
For April:(426590, 13)
For May:(604827, 13)
For June:(719618, 13)
For July:(767650, 13)
For August:(771693, 13)
For September:(666371, 13)
For October:(537113, 13)
For November:(362518, 13)
Average number of entries per file: 499618
Total number of entries across all files: 5495804


In [8]:
count_entries(data, "BikeRides_without_Duplicates.csv", True)

For January:(190301, 13)
For February:(190445, 13)
For March:(258678, 13)
For April:(426590, 13)
For May:(604827, 13)
For June:(719618, 13)
For July:(767650, 13)
For August:(771693, 13)
For September:(666371, 13)
For October:(537113, 13)
For November:(362518, 13)
Average number of entries per file: 499618
Total number of entries across all files: 5495804


From the above print out, it is clear that all the files have the same number of columns. So that is a good preliminary check on the consistency of the data across the files. And we can see that in total we have 5.5 Million entries, with an average of 500000 entries per month. All calculated numbers before and after removing duplicates are identical, so the original dataset did not have any duplicates.

Next we will look at the percentage of nulls across the columns:

In [77]:

def check_NAN(dataset, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)

        for month, df in dataset.items():
            if month == 'January':
                # if the month is January add the word "Month" to column names
                column_names = df.columns.insert(0, "Month")
                writer.writerow(column_names)
            
            percentage = df.isna().sum()*100/len(df)
            y = percentage.apply(lambda x: str(int(x))+"%" if x > 1 else ("< 1%" if x > 0 else str(0))).values
            values = np.insert(y, 0, month)
            print(values)
            writer.writerow(values)
        

In [73]:
check_NAN(data, "NaN_Percentages_Orig.csv")

['January' '0' '0' '0' '0' '0']
['February' '0' '0' '0' '0' '0']
['March' '0' '0' '0' '0' '0']
['April' '0' '0' '0' '0' '0']
['May' '0' '0' '0' '0' '0']
['June' '0' '0' '0' '0' '0']
['July' '0' '0' '0' '0' '0']
['August' '0' '0' '0' '0' '0']
['September' '0' '0' '0' '0' '0']
['October' '0' '0' '0' '0' '0']
['November' '0' '0' '0' '0' '0']


Drop the columns with NAN values, and convert the started and ended at to datetime.

In [120]:
def clean_data(dataset):
    col_to_drop = ['ride_id', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
    clean_data = {}

    for month, df in dataset.items():
        #print("shape before",df.shape)
        clean_data[month] = df.drop(columns = col_to_drop) # drop columns
        #print("shape after",df.shape)
        #df.fillna('') # fill NaN
        #df.dropnna(subset = "Column Name", inplace=True) # drop rows where column_name has NaN

        # convert columns to datetime 
        clean_data[month]['started_at'] = pd.to_datetime(clean_data[month]['started_at'])
        clean_data[month]['ended_at']   = pd.to_datetime(clean_data[month]['ended_at'])
        
    return clean_data

In [121]:
cleaned_data = clean_data(data)

In [122]:
cleaned_data['January'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190301 entries, 0 to 190300
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   rideable_type  190301 non-null  object        
 1   started_at     190301 non-null  datetime64[ns]
 2   ended_at       190301 non-null  datetime64[ns]
 3   member_casual  190301 non-null  object        
dtypes: datetime64[ns](2), object(2)
memory usage: 5.8+ MB


In [124]:
# check no of columns and entries after removing
count_entries(cleaned_data, "BikeRides_Cleaned.csv", False)

For January:(190301, 4)
For February:(190445, 4)
For March:(258678, 4)
For April:(426590, 4)
For May:(604827, 4)
For June:(719618, 4)
For July:(767650, 4)
For August:(771693, 4)
For September:(666371, 4)
For October:(537113, 4)
For November:(362518, 4)
Average number of entries per file: 499618
Total number of entries across all files: 5495804


In [125]:
check_NAN(cleaned_data, "NaN_Percentages_Clean.csv")

['January' '0' '0' '0' '0']
['February' '0' '0' '0' '0']
['March' '0' '0' '0' '0']
['April' '0' '0' '0' '0']
['May' '0' '0' '0' '0']
['June' '0' '0' '0' '0']
['July' '0' '0' '0' '0']
['August' '0' '0' '0' '0']
['September' '0' '0' '0' '0']
['October' '0' '0' '0' '0']
['November' '0' '0' '0' '0']


# Preparation:

The method does the following:
* add the column "ride_length"
* add the column "day_of_week"

In [132]:
def prepare_data(dataset):
    prepped_data = dataset

    for month, df in dataset.items():
        prepped_data[month]['ride_length'] = (df['ended_at'] - df['started_at'])#.dt.seconds
        prepped_data[month]['day_of_week'] = df['started_at'].dt.day_name()
    
    return prepped_data

In [133]:
rideleng = prepare_data(cleaned_data)
print(rideleng['January'].head())

   rideable_type          started_at            ended_at member_casual  \
0  electric_bike 2023-01-21 20:05:42 2023-01-21 20:16:33        member   
1   classic_bike 2023-01-10 15:37:36 2023-01-10 15:46:05        member   
2  electric_bike 2023-01-02 07:51:57 2023-01-02 08:05:11        casual   
3   classic_bike 2023-01-22 10:52:58 2023-01-22 11:01:44        member   
4   classic_bike 2023-01-12 13:58:01 2023-01-12 14:13:20        member   

      ride_length day_of_week  
0 0 days 00:10:51    Saturday  
1 0 days 00:08:29     Tuesday  
2 0 days 00:13:14      Monday  
3 0 days 00:08:46      Sunday  
4 0 days 00:15:19    Thursday  
